# Projeto de cruzamento de dados socio-econômicos de municípios e CNPJ's registrados pela receita e base de clientes Gyra+

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymongo
from IPython.display import clear_output
import requests as rq
from tqdm import tqdm
from unicodedata import normalize

In [2]:
def remover_acentos(txt):
        return normalize('NFKD', ''.join(filter(str.isalnum, txt))).encode('ASCII', 'ignore').decode('ASCII').lower()

## Primeiro passo: importação da base sqlite de CNPJs ativos em: https://github.com/georgevbsantiago/qsacnpj, após isso conecta-la

## Segundo passo: criar tabela de dados socio-economicos por municipio

In [3]:
# Carrega arquivo excel com os dados de pib para os municipios do brasil pra 2010 até 2018 - https://ftp.ibge.gov.br/Pib_Municipios/2018/base/base_de_dados_2010_2018_xls.zip
pib = pd.read_excel('input/PIB dos Municípios - base de dados 2010-2018.xls')
# Carrega arquivo excel com os dados de IDH para os municipios do brasil para 2010 - https://www.br.undp.org/content/brazil/pt/home/idh0/rankings/idhm-municipios-2010.html
idh = pd.read_excel('input/IDH_2010.xls')
# Carrega arquivo csv com a localizacao de todos os municipios do Brasil - https://codeload.github.com/kelvins/Municipios-Brasileiros/zip/refs/heads/main
loc = pd.read_csv('input/municipios.csv')

In [4]:
# Seleciona apenas dados de PIB para o ano de 2010
pib = pib[pib['Ano'] == 2010]
# Cria chave para o join com a base de IDH
pib.loc[:,'Cod'] = pib['Código do Município'].apply(lambda x: int(x / 10))
pib.loc[:,'Codigo IBGE'] = pib['Código do Município']
# Seleciona apenas as colunas com informações a serem usadas
pib = pib[['Sigla da Unidade da Federação', 'Nome do Município',
       'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)',
       'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)',
       'Cod', 'Codigo IBGE']]
# Renomeia colunas para mais facil entendimento
pib = pib.rename(columns={'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)': 'PIB Per Capita',
                          'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)': 'PIB'})
# Cria nova coluna de populacao estimada
pib.loc[:, 'população estimada'] = (pib['PIB'] / pib['PIB Per Capita'] * 1000).apply(int)
# Mostra o resultado final do datafram de pib por cidade
pib

,Sigla da Unidade da Federação,Nome do Município,PIB,PIB Per Capita,Cod,Codigo IBGE,população estimada
0,RO,Alta Floresta D'Oeste,2.620769e+05,10731.18,110001,1100015,24422
1,RO,Ariquemes,1.364694e+06,15103.86,110002,1100023,90353
2,RO,Cabixi,6.961111e+04,11033.62,110003,1100031,6309
3,RO,Cacoal,1.186494e+06,15095.15,110004,1100049,78601
4,RO,Cerejeiras,2.220212e+05,13037.06,110005,1100056,17030
...,...,...,...,...,...,...,...
5560,GO,Vianópolis,1.786994e+05,14240.13,522200,5222005,12549
5561,GO,Vicentinópolis,1.338994e+05,18165.70,522205,5222054,7370
5562,GO,Vila Boa,7.167577e+04,15115.09,522220,5222203,4742
5563,GO,Vila Propício,9.206286e+04,17893.66,522230,5222302,5144


In [5]:
# Seleciona apenas colunas com dados a serem utilizados
idh = idh[['Nome da Unidade da Federação', 'Código do Município', 'IDHM',
       'IDHM Educação', 'IDHM Longevidade', 'IDHM Renda']]
# Mostra o arranjo de dados de idh
idh

,Nome da Unidade da Federação,Código do Município,IDHM,IDHM Educação,IDHM Longevidade,IDHM Renda
0,Rondônia,110001,0.641,0.526,0.763,0.657
1,Rondônia,110002,0.702,0.600,0.806,0.716
2,Rondônia,110003,0.650,0.559,0.757,0.650
3,Rondônia,110004,0.718,0.620,0.821,0.727
4,Rondônia,110005,0.692,0.602,0.799,0.688
...,...,...,...,...,...,...
5559,Goiás,522200,0.712,0.622,0.818,0.710
5560,Goiás,522205,0.684,0.562,0.804,0.707
5561,Goiás,522220,0.647,0.534,0.818,0.619
5562,Goiás,522230,0.634,0.505,0.802,0.629


In [6]:
# Seleciona apenas colunas com dados a serem utilizados
loc = loc[['codigo_ibge', 'latitude', 'longitude']]
# mostra o arranjo de dados de localizacao
loc

,codigo_ibge,latitude,longitude
0,5200050,-16.75730,-49.4412
1,3100104,-18.48310,-47.3916
2,5200100,-16.19700,-48.7057
3,3100203,-19.15510,-45.4444
4,1500107,-1.72183,-48.8788
...,...,...,...
5565,2933604,-10.82300,-42.7245
5566,2517407,-8.07901,-37.1057
5567,3557154,-21.05060,-50.0552
5568,2114007,-3.27014,-45.6553


In [7]:
joined = idh.merge(pib, left_on='Código do Município', right_on='Cod', how='left').drop(columns=['Cod'])
joined = joined.set_index('Codigo IBGE')
joined = joined.drop(columns=['Código do Município'])
joined = joined.merge(loc, left_on='Codigo IBGE', right_on='codigo_ibge', how='inner')
joined

,Nome da Unidade da Federação,IDHM,IDHM Educação,IDHM Longevidade,IDHM Renda,Sigla da Unidade da Federação,Nome do Município,PIB,PIB Per Capita,população estimada,codigo_ibge,latitude,longitude
0,Rondônia,0.641,0.526,0.763,0.657,RO,Alta Floresta D'Oeste,2.620769e+05,10731.18,24422,1100015,-11.92830,-61.9953
1,Rondônia,0.702,0.600,0.806,0.716,RO,Ariquemes,1.364694e+06,15103.86,90353,1100023,-9.90571,-63.0325
2,Rondônia,0.650,0.559,0.757,0.650,RO,Cabixi,6.961111e+04,11033.62,6309,1100031,-13.49450,-60.5520
3,Rondônia,0.718,0.620,0.821,0.727,RO,Cacoal,1.186494e+06,15095.15,78601,1100049,-11.43430,-61.4562
4,Rondônia,0.692,0.602,0.799,0.688,RO,Cerejeiras,2.220212e+05,13037.06,17030,1100056,-13.18700,-60.8168
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5559,Goiás,0.712,0.622,0.818,0.710,GO,Vianópolis,1.786994e+05,14240.13,12549,5222005,-16.74050,-48.5159
5560,Goiás,0.684,0.562,0.804,0.707,GO,Vicentinópolis,1.338994e+05,18165.70,7370,5222054,-17.73220,-49.8047
5561,Goiás,0.647,0.534,0.818,0.619,GO,Vila Boa,7.167577e+04,15115.09,4742,5222203,-15.03870,-47.0520
5562,Goiás,0.634,0.505,0.802,0.629,GO,Vila Propício,9.206286e+04,17893.66,5144,5222302,-15.45420,-48.8819


In [8]:
joined.to_csv('output/dados_socio_economicos_municipios.csv', index=False)

## Terceiro passo: adicionar arquivo criado anteriormente na base de dados conectada no passo 1 e executar a seguinte query para obter os dados de cnpj e cidades relacionadas

SELECT *
FROM (SELECT *
	FROM idh_pib_municipios as ipm
	LEFT JOIN tab_codigo_municipios_siafi as tcms
	ON ipm."Codigo IBGE" == tcms.codigo_ibge
	ORDER BY IDHM) as cidades
LEFT JOIN cnpj_dados_cadastrais_pj as cdcp
ON cdcp.uf = cidades.uf and cdcp.municipio = cidades.descricao;

## Quarto passo: copiar arquivo resultado da query e base de CNPJs inedoneos para o diretório do projeto

## Quinto passo: para gerar lista de telefones e emails para prospecção 

In [9]:
# CSV criado com os dados da base de dados dos CNPJ's e dados sócio-econômicos dos municípios
df_flt = pd.read_csv('input/_SELECT_FROM_SELECT_FROM_idh_pib_municipios_as_ipm_LEFT_JOIN_tab_202105061536.csv', low_memory=False)
# CSV de CNPJ's inidôneos ou suspensos, pode ser obtido em: http://www.portaltransparencia.gov.br/download-de-dados/ceis
cnpj_ruim = pd.read_csv('input/20210512_CEIS.csv', delimiter=';')

In [10]:
# Seleciona apenas CNPJ's que a data de inicio das atividades seja mais antiga que 300 dias (aproximadamente 10 meses)
df_flt = df_flt[df_flt['data_inicio_atividade'].apply(pd.Timestamp) < pd.Timestamp(pd.Timestamp.today() - pd.Timedelta('300D'))]

In [11]:
# Seleciona apenas CNPJ's que sejam matrizes, excluindo assim filiais de empresas grandes, como por exemplo: bancos, correios, hypermercados e etc.
df_flt = df_flt[df_flt['identificador_matriz_filial'] == 1]

In [12]:
# Realiza o processamento dos dados telefonicos, valida o comprimento e checa se o valor é um telefone fixo ou celular
df_flt.loc[df_flt.loc[:, 'ddd_telefone_1'] != df_flt.loc[:, 'ddd_telefone_1'], 'ddd_telefone_1'] = '0'
df_flt.loc[:, 'ddd_telefone_1'] = df_flt['ddd_telefone_1'].apply(lambda x: ''.join(filter(str.isdigit, x)))
df_flt.loc[df_flt['ddd_telefone_1'].str[0] == '0','ddd_telefone_1'] = df_flt['ddd_telefone_1'][df_flt['ddd_telefone_1'].str[0] == '0'].str[1:]
df_flt.loc[df_flt['ddd_telefone_1'] == '', 'ddd_telefone_1'] = '0'
df_flt['1digito'] = df_flt['ddd_telefone_1'].str[-8:-7]
df_flt.loc[df_flt['1digito'] == '', '1digito'] = '0'
df_flt.loc[:, '1digito'] = df_flt['1digito'].apply(int)
df_flt.loc[df_flt['1digito'] < 6, 'ddd_telefone_1'] = '0'
df_flt.loc[df_flt['ddd_telefone_1'] == '0', 'ddd_telefone_1'] = np.nan
df_flt = df_flt.drop(columns=['1digito'])
df_flt.loc[df_flt['ddd_telefone_1'].str.len() < 10, 'ddd_telefone_1'] = np.nan

In [13]:
# Seleciona apenas entidades empresariais, descartando orgão nao-lucrativos, como organizacoes governamentais ou nao governamentais
df_flt = df_flt[df_flt['codigo_natureza_juridica'].apply(str).str[0] == '2']

In [14]:
# Exclue CNAE's de empresas com categorias de trabalho que nao podemos emprestar
df_flt = df_flt[df_flt['cnae_fiscal'] != 9492800.0]

In [15]:
# Descarta CNPJ's com registro na base de inidoneos ou suspensos
df_flt = df_flt[~df_flt['cnpj.1'].isin(cnpj_ruim['CPF OU CNPJ DO SANCIONADO'].values)]

In [16]:
# Seleciona apenas os campos relevantes para cria a audiencia
df_flt = df_flt[['ddd_telefone_1', 'correio_eletronico', 'uf', 'descricao', 'IDHM', 'cnpj.1']]

In [17]:
# Tratamento dos valores de email
df_flt.loc[~df_flt['correio_eletronico'].str.contains('@').fillna(False), 'correio_eletronico'] = np.nan
df_flt.loc[:,'correio_eletronico'] = df_flt['correio_eletronico'].str.lower()

In [18]:
# Seleciona apenas CNPJ's que tenham ou telefone ou email
df_flt = df_flt[np.logical_or(df_flt['ddd_telefone_1'] == df_flt['ddd_telefone_1'], df_flt['correio_eletronico'] == df_flt['correio_eletronico'])]

In [19]:
df_flt.to_csv('output/base_telefone_email.csv', index=False)

# Tratamento dos dados da collection e dados socio-economicos para juncao

In [20]:
# Cria a conexao com o mongo
username = input('digite seu usuário:')
password = input('digite sua senha:')
clear_output()
client = pymongo.MongoClient('mongodb+srv://' + username + ':' + password +
                             '@gyramais-production.lhwtb.mongodb.net/myFirstDatabase?retryWrites=true&readPreference=secondary&readPreferenceTags=nodeType:ANALYTICS&w=majority')
del username, password
# Coleta todos os documentos da collection address
address = list(client.gyramais.Address.find())
# Cria o dataframe desses documentos
df_address = pd.DataFrame(address)
# Mostra a disposição dos dados do dataframe
df_address

,_id,street,complement,number,city,_p_business,district,state,postalCode,_wperm,_rperm,_acl,_created_at,_updated_at,_p_proofOfAddress,moneyPlusId,_p_user
0,Kx2C8vJf1g,Rua Engenheiro Haroldo Cavalcanti,sala 305,360,Rio de Janeiro,Business$hQHd1tmaFF,Recreio dos Bandeirantes,RJ,22795240,"[role:superuser, role:owner-hQHd1tmaFF, role:a...","[role:superuser, role:administrator, role:owne...","{'role:superuser': {'w': True, 'r': True}, 'ro...",2019-12-12 19:03:35.994,2020-08-06 19:16:53.955,File$2MXOgpCMUF,NaN,NaN
1,HGMv2uYe9H,Rua Jamelão,(Prq Árvores (FEI-MIZUHO),129,São Bernardo do Campo,Business$ocMPk8yEtG,Alves Dias,SP,09855600,"[role:superuser, role:owner-ocMPk8yEtG, role:a...","[role:superuser, role:administrator, role:owne...","{'role:superuser': {'w': True, 'r': True}, 'ro...",2019-12-12 19:03:38.308,2020-06-23 23:02:46.515,File$Qsl3PXJjbG,NaN,NaN
2,0m66actqZJ,Rodovia galerano augusto venturini,,2010,Santa Maria de Jetibá,Business$HlnS59xf0z,SAO LUIS,ES,29645000,"[role:superuser, role:owner-HlnS59xf0z, role:a...","[role:superuser, role:administrator, role:owne...","{'role:superuser': {'w': True, 'r': True}, 'ro...",2019-12-12 19:03:40.325,2020-04-29 13:58:35.987,File$MIk3NvmNjV,NaN,NaN
3,CWobva1xRD,Rua Ática,Conjunto 72B,535,São Paulo,Business$n7aHzbGb7A,Jardim Brasil (Zona Sul),SP,04634042,"[role:superuser, role:owner-n7aHzbGb7A, role:a...","[role:superuser, role:administrator, role:owne...","{'role:superuser': {'w': True, 'r': True}, 'ro...",2019-12-12 19:03:42.303,2020-05-05 20:02:26.649,File$iKVywhMCl6,NaN,NaN
4,I3S2r9s9U7,José Maria de Souza,,868,São Pedro,Business$xyauxtyUKQ,São Tomé,SP,13520000,"[role:superuser, role:owner-xyauxtyUKQ, role:a...","[role:superuser, role:administrator, role:owne...","{'role:superuser': {'w': True, 'r': True}, 'ro...",2019-12-12 19:03:44.146,2019-12-12 19:03:44.146,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178161,D6FTd8S6FX,R TIA NADIR,,284,BELO HORIZONTE,Business$AtHejUHXGU,VISTA DO SOL,MG,31990550,"[role:superuser, role:owner-AtHejUHXGU, role:a...","[role:superuser, role:administrator, role:owne...","{'role:superuser': {'w': True, 'r': True}, 'ro...",2021-05-18 15:18:11.896,2021-05-18 15:18:11.896,NaN,NaN,NaN
178162,K5kmE7XiyG,TV DO SAL,,22,RIO DE JANEIRO,Business$6hEnT8H6tv,CIDADE DE DEUS,RJ,22773080,"[role:superuser, role:owner-6hEnT8H6tv, role:a...","[role:superuser, role:administrator, role:owne...","{'role:superuser': {'w': True, 'r': True}, 'ro...",2021-05-18 15:20:13.260,2021-05-18 15:20:13.260,NaN,NaN,NaN
178163,HL2Va0Oj3i,R DOURADA (CJ HAB BRASILANDIA B),BLOCO,8,SAO PAULO,Business$uEAdGKAdo4,VILA BRASILANDIA,SP,02820090,"[role:superuser, role:owner-uEAdGKAdo4, role:a...","[role:superuser, role:administrator, role:owne...","{'role:superuser': {'w': True, 'r': True}, 'ro...",2021-05-18 15:20:13.453,2021-05-18 15:20:13.453,NaN,NaN,NaN
178164,fFtTxgMHpG,R RUA SANTA EDWIRGES,CASA,06,SAO GONCALO DO AMARANTE,Business$6zYfFXvpwM,NOVA TAIBA,CE,62670000,"[role:superuser, role:owner-6zYfFXvpwM, role:a...","[role:superuser, role:administrator, role:owne...","{'role:superuser': {'w': True, 'r': True}, 'ro...",2021-05-18 15:23:12.379,2021-05-18 15:23:12.379,NaN,NaN,NaN


In [21]:
# Transforma nome das cidades em padrao americano
df_address.loc[:, 'city_mod'] = df_address['city'].apply(lambda x:remover_acentos(x)) + '-' + df_address['state'].str.lower()
# Transforma nome das cidades em padrão americano
joined.loc[:, 'city_mod'] = joined['Nome do Município'].apply(lambda x: remover_acentos(x)) + '-' + joined['Sigla da Unidade da Federação'].str.lower()

In [22]:
left = pd.merge(left=df_address, right=joined, how='left', left_on='city_mod', right_on='city_mod')[['_id', 'codigo_ibge']].set_index('_id')

In [23]:
left.to_csv('output/collection_address_id_codigo_ibge.csv')